In [10]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

data = pd.read_csv('/kaggle/input/mlpr-split-encoded-data/holdout (1).csv')

# 1) Storm Forecasting

In [11]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.models import load_model

storm_features = [
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'duration_hours',
    'desc_word_count', 'has_tornado', 'has_hail', 'has_flood', 'has_wind',
    'has_tree', 'has_broken', 'has_blown', 'tmin', 'tmax', 'tavg', 'ppt',
    'MAGNITUDE_IMPUTED', 'STATE_FIPS'
]
target_col = 'is_storm_lagged'
max_seq_length = 6

storm_model = joblib.load('/kaggle/input/storm_lstm_model/tensorflow2/default/1/storm_lstm_model (1).pkl')
scaler = joblib.load('/kaggle/input/storm_lstm_model/tensorflow2/default/1/storm_scaler (4).pkl')

def create_sequences(df, features, max_seq_length):
    X, y, indices = [], [], []
    for state in df['st_abb'].unique():
        state_df = df[df['st_abb'] == state].sort_values('power_outage_datetime')
        state_features = state_df[features].values
        state_target = state_df[target_col].values
        for i in range(len(state_df)):
            seq_start = max(0, i - max_seq_length + 1)
            seq_data = state_features[seq_start:i + 1]
            seq_len = len(seq_data)
            if seq_len == 0:
                continue
            padded_seq = np.zeros((max_seq_length, len(features)))
            padded_seq[-seq_len:] = seq_data
            X.append(padded_seq)
            y.append(state_target[i])
            indices.append(state_df.index[i])
    return np.array(X), np.array(y), indices

X_seq, y_seq, idx_seq = create_sequences(data, storm_features, max_seq_length)

# Scale the data
X_seq_reshaped = X_seq.reshape(-1, len(storm_features))
X_seq_scaled = scaler.transform(X_seq_reshaped).reshape(X_seq.shape)

storm_probs = storm_model.predict(X_seq_scaled)
storm_preds = (storm_probs > 0.5).astype(int).flatten()

data['predicted_storm'] = 0  # Initialize
data.loc[idx_seq, 'predicted_storm'] = storm_preds

# ===============================
# Evaluate model
# ===============================
y_true = data.loc[idx_seq, target_col]
y_pred = data.loc[idx_seq, 'predicted_storm']

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1_storm = f1_score(y_true, y_pred, zero_division=0)
conf_matrix = confusion_matrix(y_true, y_pred)

print("Storm Prediction Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_storm:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

storm_data = data[data['predicted_storm'] == 1].copy()

1315/1315 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Storm Prediction Metrics:
Accuracy : 0.8981
Precision: 0.9133
Recall   : 0.8853
F1 Score : 0.8991

Confusion Matrix:
[[18681  1812]
 [ 2473 19089]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.91      0.90     20493
           1       0.91      0.89      0.90     21562

    accuracy                           0.90     42055
   macro avg       0.90      0.90      0.90     42055
weighted avg       0.90      0.90      0.90     42055



# 2) Severity Prediction

In [12]:
# ----------------- SEVERITY PREDICTION -----------------

# Define severity features
severity_features = [
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'duration_hours',
    'desc_word_count', 'has_tornado', 'has_hail', 'has_flood', 'has_wind',
    'has_tree', 'has_broken', 'has_blown', 'tmin', 'tmax', 'tavg',
    'EVENT_TYPE_encoded', 'stability_encoded', 'CZ_FIPS', 'WFO_encoded'
]
target_col = 'severity_class'

# Load severity model and scaler
severity_model = joblib.load('/kaggle/input/severity_lgb_model-1/scikitlearn/default/1/severity_lgb_model (1).pkl')
severity_scaler = joblib.load('/kaggle/input/severity_lgb_model-1/scikitlearn/default/1/severity_scaler (3).pkl')

# Prepare features and scale
X = storm_data[severity_features].values
X_scaled = severity_scaler.transform(X)

# Severity prediction
severity_preds = severity_model.predict(X_scaled)
storm_data['predicted_severity'] = severity_preds

# Evaluate severity model
y_true = storm_data[target_col]
y_pred = storm_data['predicted_severity']
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1_severity = f1_score(y_true, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_true, y_pred)

severity_counts = y_true.value_counts().sort_index()  # Counts of true Low (0), Medium (1), High (2)
support_low = severity_counts.get(0, 0)
support_medium = severity_counts.get(1, 0)
support_high = severity_counts.get(2, 0)
total_support = support_low + support_medium + support_high

print("Severity Prediction Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_severity:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

# Filter severity cases
low_severity_data = storm_data[storm_data['predicted_severity'] == 0].copy()
med_severity_data = storm_data[storm_data['predicted_severity'] == 1].copy()
high_severity_data = storm_data[storm_data['predicted_severity'] == 2].copy()

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Severity Prediction Metrics:
Accuracy : 0.9196
Precision: 0.9198
Recall   : 0.9196
F1 Score : 0.9197

Confusion Matrix:
[[6606  177   93    0]
 [ 140 6040  630    0]
 [  69  565 6493    0]
 [   6    0    0   82]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      6876
           1       0.89      0.89      0.89      6810
           2       0.90      0.91      0.91      7127
          10       1.00      0.93      0.96        88

    accuracy                           0.92     20901
   macro avg       0.94      0.92      0.93     20901
weighted avg       0.92      0.92      0.92     20901



# 3) Outage Prediction

In [13]:
import numpy as np

# Derive is_outage target (assuming customers_out > 0 indicates an outage)
for df in [low_severity_data, med_severity_data, high_severity_data]:
    df['is_outage'] = (df['customers_out'] > 0).astype(int)

# Extract non-storm rows from storm_df_with_predictions
non_storm_data = data[data['predicted_storm'] == 0].copy()
non_storm_data['is_outage'] = 0  # No outage for non-storm events
non_storm_data['predicted_severity'] = 10  # Placeholder for non-storm rows

# Randomly split non-storm rows across the three DataFrames
non_storm_split = np.array_split(non_storm_data.sample(frac=1, random_state=42), 3)
non_storm_low, non_storm_medium, non_storm_high = non_storm_split

# Augment DataFrames with non-storm rows
low_severity_data = pd.concat([low_severity_data, non_storm_low], ignore_index=True)
med_severity_data = pd.concat([med_severity_data, non_storm_medium], ignore_index=True)
high_severity_data = pd.concat([high_severity_data, non_storm_high], ignore_index=True)

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
outage_features = [
    'tmin', 'tmax', 'tavg', 'stability_encoded',
    'EVENT_TYPE_encoded', 'duration_hours', 'desc_word_count',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'has_broken', 'has_blown', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
    'INJURIES_DIRECT', 'DEATHS_DIRECT', 'CZ_FIPS'
]
target_col = 'is_outage'

### a. Low

In [15]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ----------------- LOAD MODEL AND SCALER -----------------
outage_model = joblib.load('/kaggle/input/low_severity_lgm_model-1/scikitlearn/default/1/low_severity_lgm_model (1).pkl')
scaler = joblib.load('/kaggle/input/low_severity_lgm_model-1/scikitlearn/default/1/low_severity_scaler (3).pkl')

# ----------------- PREPARE FEATURES -----------------
X = low_severity_data[outage_features]

# ----------------- SCALE FEATURES -----------------
X_scaled = scaler.transform(X)

# ----------------- SEVERITY PREDICTION -----------------
outage_pred = outage_model.predict(X_scaled)

# ----------------- TRUE AND PREDICTED LABELS -----------------
target_col = 'is_outage'  # Assuming this is the target column based on prior context
y_true_outage = low_severity_data[target_col]
y_pred_outage = outage_pred

# ----------------- METRIC CALCULATIONS -----------------
accuracy = accuracy_score(y_true_outage, y_pred_outage)
precision = precision_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
recall = recall_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
f1_low = f1_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
conf_matrix = confusion_matrix(y_true_outage, y_pred_outage)

# ----------------- OUTPUT -----------------
print("Outage Prediction Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_low:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true_outage, y_pred_outage, zero_division=0))

Outage Prediction Metrics:
Accuracy : 0.9850
Precision: 0.9891
Recall   : 0.9801
F1 Score : 0.9846

Confusion Matrix:
[[7010   73]
 [ 135 6655]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      7083
           1       0.99      0.98      0.98      6790

    accuracy                           0.99     13873
   macro avg       0.99      0.98      0.98     13873
weighted avg       0.99      0.99      0.99     13873



### b. Medium

In [16]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ----------------- LOAD MODEL AND SCALER -----------------
outage_model = joblib.load('/kaggle/input/medium_severity_xgb_model/scikitlearn/default/1/medium_severity_xgb_model.pkl')
scaler = joblib.load('/kaggle/input/medium_severity_xgb_model/scikitlearn/default/1/medium_severity_scaler (1).pkl')

# ----------------- PREPARE FEATURES -----------------
X = med_severity_data[outage_features]

# ----------------- SCALE FEATURES -----------------
X_scaled = scaler.transform(X)

# ----------------- SEVERITY PREDICTION -----------------
outage_pred = outage_model.predict(X_scaled)

# ----------------- TRUE AND PREDICTED LABELS -----------------
target_col = 'is_outage'  # Assuming this is the target column based on prior context
y_true_outage = med_severity_data[target_col]
y_pred_outage = outage_pred

# ----------------- METRIC CALCULATIONS -----------------
accuracy = accuracy_score(y_true_outage, y_pred_outage)
precision = precision_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
recall = recall_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
f1_medium = f1_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
conf_matrix = confusion_matrix(y_true_outage, y_pred_outage)

# ----------------- OUTPUT -----------------
print("Outage Prediction Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_medium:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true_outage, y_pred_outage, zero_division=0))

Outage Prediction Metrics:
Accuracy : 0.9775
Precision: 0.9884
Recall   : 0.9654
F1 Score : 0.9768

Confusion Matrix:
[[6985   77]
 [ 234 6537]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7062
           1       0.99      0.97      0.98      6771

    accuracy                           0.98     13833
   macro avg       0.98      0.98      0.98     13833
weighted avg       0.98      0.98      0.98     13833



### c. High

In [17]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ----------------- LOAD MODEL AND SCALER -----------------
outage_model = joblib.load('/kaggle/input/high_severity_rf_model-2/scikitlearn/default/1/high_severity_rf_model (2).pkl')
scaler = joblib.load('/kaggle/input/high_severity_rf_model-2/scikitlearn/default/1/high_severity_scaler (2).pkl')

# ----------------- PREPARE FEATURES -----------------
X = high_severity_data[outage_features]

# ----------------- SCALE FEATURES -----------------
X_scaled = scaler.transform(X)

# ----------------- SEVERITY PREDICTION -----------------
outage_pred = outage_model.predict(X_scaled)

# ----------------- TRUE AND PREDICTED LABELS -----------------
target_col = 'is_outage'  # Assuming this is the target column based on prior context
y_true_outage = high_severity_data[target_col]
y_pred_outage = outage_pred

# ----------------- METRIC CALCULATIONS -----------------
accuracy = accuracy_score(y_true_outage, y_pred_outage)
precision = precision_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
recall = recall_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
f1_high = f1_score(y_true_outage, y_pred_outage, average='binary', zero_division=0)
conf_matrix = confusion_matrix(y_true_outage, y_pred_outage)

# ----------------- OUTPUT -----------------
print("Outage Prediction Metrics (High Severity):")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1_high:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_true_outage, y_pred_outage, zero_division=0))

Outage Prediction Metrics (High Severity):
Accuracy : 0.9694
Precision: 0.9846
Recall   : 0.9543
F1 Score : 0.9692

Confusion Matrix:
[[6968  107]
 [ 329 6863]]

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      7075
           1       0.98      0.95      0.97      7192

    accuracy                           0.97     14267
   macro avg       0.97      0.97      0.97     14267
weighted avg       0.97      0.97      0.97     14267



In [18]:
# Compute weights based on true severity supports
w_low = support_low / total_support
w_medium = support_medium / total_support
w_high = support_high / total_support

# Compute the overall evaluation metric
overall_metric = f1_storm * f1_severity * (w_low * f1_low + w_medium * f1_medium + w_high * f1_high)

# Print the results
print("\nOverall Model Evaluation:")
print(f"F1 Storm: {f1_storm:.4f}")
print(f"F1 Severity (macro): {f1_severity:.4f}")
print(f"Weights - Low: {w_low:.3f}, Medium: {w_medium:.3f}, High: {w_high:.3f}")
print(f"F1 Low: {f1_low:.4f}, F1 Medium: {f1_medium:.4f}, F1 High: {f1_high:.4f}")
print(f"Overall Metric (F1_storm * F1_severity * (wL*F1L + wM*F1M + wH*F1H)): {overall_metric:.4f}")


Overall Model Evaluation:
F1 Storm: 0.8991
F1 Severity (macro): 0.9197
Weights - Low: 0.330, Medium: 0.327, High: 0.342
F1 Low: 0.9846, F1 Medium: 0.9768, F1 High: 0.9692
Overall Metric (F1_storm * F1_severity * (wL*F1L + wM*F1M + wH*F1H)): 0.8077
